In [1]:
import cloudscraper as cs
import bs4
import json
import datetime
import pickle
import requests
import os
import numpy

### Helpers

In [2]:
def parse_asset_data(asset_dict):
    
    asset_id = asset_dict['token_id']
    
    try:
        creator_username = asset_dict['creator']['user']['username']
    except:
        creator_username = None
    try:
        creator_address = asset_dict['creator']['address']
    except:
        creator_address = None
    
    try:
        owner_username = asset_dict['owner']['user']['username']
    except:
        owner_username = None
    
    owner_address = asset_dict['owner']['address']
    
    traits = asset_dict['traits']
    num_sales = int(asset_dict['num_sales'])
        
    result = {'asset_id': asset_id,
              'creator_username': creator_username,
              'creator_address': creator_address,
              'owner_username': owner_username,
              'owner_address': owner_address,
              'traits': traits,
              'num_sales': num_sales}
    
    return result


def parse_sale_data(sale_dict):
    
    is_bundle = False

    if sale_dict['asset'] != None:
        asset_id = sale_dict['asset']['token_id']
    elif sale_dict['asset_bundle'] != None:
        asset_id = [asset['token_id'] for asset in sale_dict['asset_bundle']['assets']]
        is_bundle = True
    
    
    seller_address = sale_dict['seller']['address']
    buyer_address = sale_dict['winner_account']['address']
    
    try:
        seller_username = sale_dict['seller']['user']['username']
    except:
        seller_username = None    
    try:
        buyer_username = sale_dict['winner_account']['user']['username']
    except:
        buyer_username = None
    
    timestamp = sale_dict['transaction']['timestamp']
    total_price = float(sale_dict['total_price'])
    payment_token = sale_dict['payment_token']['symbol']
    usd_price = float(sale_dict['payment_token']['usd_price'])
    transaction_hash = sale_dict['transaction']['transaction_hash']
    

    result = {'is_bundle': is_bundle,
              'asset_id': asset_id,
              'seller_address': seller_address,
              'buyer_address': buyer_address,
              'buyer_username': buyer_username,
              'seller_username':seller_username,
              'timestamp': timestamp,
              'total_price': total_price, 
              'payment_token': payment_token,
              'usd_price': usd_price,
              'transaction_hash': transaction_hash}
    
    return result

In [3]:
import csv

def writeCSV(fname, data):
    with open(fname, 'w', newline='') as file:
        writer = csv.writer(file)
        
        for el in data:
            writer.writerow(el)

In [4]:
with open(f"{os.getcwd()}/opensea-api-key") as f:
    OS_API_KEY = f.read()[:-2]

# Get Slugs

In [5]:
files = [f for f in os.listdir() if f[-4:] == ".pkl"] 
print(files)

data = {}
for file in files:
    with open(file, 'rb') as f:
        loaded_file = pickle.load(f)
        data[file] = loaded_file

['25-02-2022_11-10-56_rankings.pkl']


In [6]:
d = data[files[0]]

In [7]:
d

{'3landers': {'__id': 'Q29sbGVjdGlvblR5cGU6NzY1MTU3MA==',
  '__typename': 'CollectionType',
  'createdDate': '2022-02-19T13:01:18.111966',
  'name': '3Landers',
  'slug': '3landers',
  'logo': 'https://lh3.googleusercontent.com/jM8itRiSM3hI8RWokMkhR97JdoIzukU3DUEO-9MD3i6r6DnTBW6efbv1617zES0MfoiWzW4f3UaL9lgRr61vf3Cj4DDPIsk_XjNBXQ=s120',
  'isVerified': False,
  'nativePaymentAsset': {'__ref': 'UGF5bWVudEFzc2V0VHlwZTo0Mg=='},
  'statsV2': {'__ref': 'client:Q29sbGVjdGlvblR5cGU6NzY1MTU3MA==:statsV2'},
  'id': 'Q29sbGVjdGlvblR5cGU6NzY1MTU3MA=='},
 'tubby-cats': {'__id': 'Q29sbGVjdGlvblR5cGU6Nzc1MDIzNQ==',
  '__typename': 'CollectionType',
  'createdDate': '2022-02-21T16:56:00.741611',
  'name': 'tubby cats by tubby collective',
  'slug': 'tubby-cats',
  'logo': 'https://lh3.googleusercontent.com/XHZY9623keDQqFSDHKqOdcjD99Y7N82K1egYRM2Mm1Z-Jxn5myrkKiC5NBktWKStVtTzDzwELy9dNpzTWJTIkLsdMIxUHI86jduQ=s120',
  'isVerified': False,
  'nativePaymentAsset': {'__ref': 'UGF5bWVudEFzc2V0VHlwZTo0Mg=='},


# Pull assets

In [ ]:
orders = ["sales_date", "sale_count", "sale_price", "pk"]

url = "https://api.opensea.io/api/v1/assets"

def pullAssets(slug):
    data = []

    for i in range(0, 400):
        headers = {'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/97.0.4692.71 Safari/537.36','referrer':url, "X-Api-Key": OS_API_KEY}#config["OPENSEA_API_KEY"]}
        querystring = {#"token_ids":list(range((i*50)+1, (i*50)+51)),
                       "collection": slug,
                       "order_direction":"desc",
                       "limit":"50"}
        response = requests.get(url, params=querystring, headers=headers)

        print(i, end=" ")
        if response.status_code != 200:
            print(response.status_code)
            print('error')
            break

        #Getting meebits data
        assets = response.json()['assets']
        #Parsing meebits data
        parsed_assets = [parse_asset_data(asset) for asset in assets]
        #storing parsed meebits data into MongoDB
        data.append(parsed_assets)
        
    return data

In [10]:
for k in d.keys():
    print(k)
    pullAssets(k)
    print()

3landers
0 401
error

tubby-cats
0 401
error

invisiblefriends
0 401
error

boredapeyachtclub
0 401
error

mfers
0 401
error

nft-worlds
0 401
error

azuki
0 401
error

catbloxgenesis
0 401
error

clonex
0 401
error

mutant-ape-yacht-club
0 401
error

worldwidewebbland
0 401
error

snoop-dogg-doggies
0 401
error

cryptopunks
0 401
error

starcatchersnft
0 401
error

rtfkt-mnlth
0 401
error

edenhorde-official


KeyboardInterrupt: 

In [38]:
d

{'3landers': {'__id': 'Q29sbGVjdGlvblR5cGU6NzY1MTU3MA==',
  '__typename': 'CollectionType',
  'createdDate': '2022-02-19T13:01:18.111966',
  'name': '3Landers',
  'slug': '3landers',
  'logo': 'https://lh3.googleusercontent.com/jM8itRiSM3hI8RWokMkhR97JdoIzukU3DUEO-9MD3i6r6DnTBW6efbv1617zES0MfoiWzW4f3UaL9lgRr61vf3Cj4DDPIsk_XjNBXQ=s120',
  'isVerified': False,
  'nativePaymentAsset': {'__ref': 'UGF5bWVudEFzc2V0VHlwZTo0Mg=='},
  'statsV2': {'__ref': 'client:Q29sbGVjdGlvblR5cGU6NzY1MTU3MA==:statsV2'},
  'id': 'Q29sbGVjdGlvblR5cGU6NzY1MTU3MA=='},
 'tubby-cats': {'__id': 'Q29sbGVjdGlvblR5cGU6Nzc1MDIzNQ==',
  '__typename': 'CollectionType',
  'createdDate': '2022-02-21T16:56:00.741611',
  'name': 'tubby cats by tubby collective',
  'slug': 'tubby-cats',
  'logo': 'https://lh3.googleusercontent.com/XHZY9623keDQqFSDHKqOdcjD99Y7N82K1egYRM2Mm1Z-Jxn5myrkKiC5NBktWKStVtTzDzwELy9dNpzTWJTIkLsdMIxUHI86jduQ=s120',
  'isVerified': False,
  'nativePaymentAsset': {'__ref': 'UGF5bWVudEFzc2V0VHlwZTo0Mg=='},
